In [53]:
# linear algebra 
import numpy as np 
# data processing, CSV file I / O (e.g. pd.read_csv) 
import pandas as pd 
from tensorflow import keras
import os 
import tensorflow as tf 
from keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from keras.models import Model 
from keras.layers import Flatten, Dense, LSTM, Dropout, Embedding, Activation 
from keras.layers import concatenate, BatchNormalization, Input
from keras.layers import add 
from keras.utils import to_categorical, plot_model 
from keras.applications.inception_v3 import InceptionV3, preprocess_input 
import matplotlib.pyplot as plt # for plotting data 
import cv2
print("Done importing")

Done importing


In [54]:
cwd = "/home/professor/Downloads/fromgit/archive"

In [55]:
reports_path = "/home/professor/Downloads/fromgit/archive/indiana_reports.csv"
reports_df = pd.read_csv(reports_path)
reports_df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


In [56]:
projections_path="/home/professor/Downloads/fromgit/archive/indiana_projections.csv"
projections_df = pd.read_csv(projections_path)
projections_df.head()

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal


In [57]:
merged_df = pd.merge(projections_df ,reports_df , on ="uid" )

In [58]:
merged_df

,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,1,1_IM-0001-3001.dcm.png,Lateral,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
2,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
3,2,2_IM-0652-2001.dcm.png,Lateral,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
4,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
...,...,...,...,...,...,...,...,...,...,...
7461,3997,3997_IM-2048-1002.dcm.png,Lateral,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB."
7462,3998,3998_IM-2048-1001.dcm.png,Frontal,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.
7463,3998,3998_IM-2048-1002.dcm.png,Lateral,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.
7464,3999,3999_IM-2049-1001.dcm.png,Frontal,normal,normal,"CHEST PA and LATERAL: on XXXX, XXXX.",This is a XXXX-year-old female patient with sh...,"Chest x-XXXX, XXXX, XXXX.",NaN,The cardiac silhouette is normal in size and c...


In [59]:
final_df = merged_df[merged_df['projection'] != 'Lateral' ]

In [60]:
final_df

,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
2,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
4,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
6,4,4_IM-2050-1001.dcm.png,Frontal,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
8,5,5_IM-2117-1003002.dcm.png,Frontal,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.
...,...,...,...,...,...,...,...,...,...,...
7456,3995,3995_IM-2046-1001.dcm.png,Frontal,Lung/hyperdistention/mild;Diaphragm/bilateral/...,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,Xray Chest PA and Lateral,"Nausea, vomiting x2 weeks. Dialysis patient.","XXXX, XXXX.",The cardiomediastinal silhouette and pulmonary...,1. Interval resolution of bibasilar airspace d...
7458,3996,3996_IM-2047-1001.dcm.png,Frontal,Spine/degenerative,Spine,Xray Chest PA and Lateral,NaN,None.,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...
7460,3997,3997_IM-2048-1001.dcm.png,Frontal,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB."
7462,3998,3998_IM-2048-1001.dcm.png,Frontal,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.


In [61]:
result_dict = dict(zip(final_df['filename'], final_df['findings']))
result_dict

{'1_IM-0001-4001.dcm.png': 'The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.',
 '2_IM-0652-1001.dcm.png': 'Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX.',
 '3_IM-1384-1001.dcm.png': nan,
 '4_IM-2050-1001.dcm.png': 'There are diffuse bilateral interstitial and alveolar opacities consistent with chronic obstructive lung disease and bullous emphysema. There are irregular opacities in the left lung apex, that could represent a cavitary lesion in the left lung apex.There are streaky opacities in the right upper lobe, XXXX scarring. The cardiomediastinal silhouette is normal in size and contour. There is no pneumothorax or large pleural effusion.',
 '5_IM-2117-1003002.dcm.png': 'The cardiomediastinal silhouette and pulmonary vasculature are within normal limits. 

In [62]:
import string
def clean_captions(captions): 
    keys_to_remove = []
    for key, caption in captions.items():
        if  isinstance(caption, str): 
            caption = [ch for ch in caption if ch not in string.punctuation] 
            caption = ''.join(caption) 
            caption = caption.split(' ') 
            caption = [word.lower() for word in caption if len(word)>1 and word.isalpha()] 
            caption = ' '.join(caption) 
        else:
            keys_to_remove.append(key)
    for key in keys_to_remove:
        del captions[key]
    return captions
result_dict = clean_captions(result_dict) 
result_dict

{'1_IM-0001-4001.dcm.png': 'The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.',
 '2_IM-0652-1001.dcm.png': 'Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX.',
 '4_IM-2050-1001.dcm.png': 'There are diffuse bilateral interstitial and alveolar opacities consistent with chronic obstructive lung disease and bullous emphysema. There are irregular opacities in the left lung apex, that could represent a cavitary lesion in the left lung apex.There are streaky opacities in the right upper lobe, XXXX scarring. The cardiomediastinal silhouette is normal in size and contour. There is no pneumothorax or large pleural effusion.',
 '5_IM-2117-1003002.dcm.png': 'The cardiomediastinal silhouette and pulmonary vasculature are within normal limits. There is no pneumothorax or pleu

In [63]:
len(result_dict)

3307

In [64]:
def to_vocab(captions): 
    words = set() 
    for key,caption in captions.items():  
        words.update(caption.split()) 
    return words 
vocab = to_vocab(result_dict)
vocab

{'Residuals',
 'retrosternal',
 'mitral',
 'retained',
 'clothing',
 'which',
 'silhouettes.',
 'Pleural',
 'limit',
 'segment.',
 'Surgical',
 'Remainder',
 'vascularity',
 'Interstitial',
 'enthesophyte.',
 'cyst',
 'frontal',
 'airspace.',
 'examinations',
 'hilum.',
 'Question',
 'Considering',
 'either',
 'medication',
 'Appearance',
 'XXXX;',
 'whose',
 'vertebroplasty',
 'near',
 'prosthesis.',
 'hyperinflation,',
 'basilar',
 'aortic',
 'healed',
 'unchanged',
 'noted.',
 'prominence,',
 'Two-view',
 'Hindfoot,',
 'limited',
 '4th,',
 'postsurgical,',
 'densities.',
 'your',
 'Lordotic',
 'broken,',
 'lobulated',
 'L1-L2.',
 'superior-most',
 'superimposition',
 'thickening.',
 'deformities,',
 'wedge-shaped',
 'fragments',
 'shortening',
 'pads,',
 '1.7',
 'consolidated.',
 'feeding',
 'thickening,',
 'fractures',
 'described',
 'nodule,',
 'After',
 'consists',
 'Since',
 'lucency.',
 'degeneration',
 'Dual-lumen',
 'bronchial',
 'thoracolumbar',
 'x-XXXX',
 'questionable',
 

In [65]:
len(vocab)

2713

In [66]:
len(result_dict.keys())

3307

In [67]:
images_path = cwd +'/images/images_normalized/'
img_paths = []
for key in result_dict.keys():
    img_paths.append(images_path + key)
img_paths
print(len(img_paths))


3307


In [68]:
img_paths

['/home/professor/Downloads/fromgit/archive/images/images_normalized/1_IM-0001-4001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/2_IM-0652-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/4_IM-2050-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/5_IM-2117-1003002.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/6_IM-2192-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/7_IM-2263-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/8_IM-2333-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/9_IM-2407-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/10_IM-0002-2001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/images_normalized/11_IM-0067-1001.dcm.png',
 '/home/professor/Downloads/fromgit/archive/images/imag

In [69]:
train_images_paths = img_paths[:3000]
val_images_paths = img_paths[3000:3400]
test_images_paths = img_paths[3400:3788]
len(train_images_paths), len(val_images_paths), len(test_images_paths)

(3000, 307, 0)

In [70]:
train_img_names = [ imgname[len(images_path):] for imgname in train_images_paths]
train_img_names

['1_IM-0001-4001.dcm.png',
 '2_IM-0652-1001.dcm.png',
 '4_IM-2050-1001.dcm.png',
 '5_IM-2117-1003002.dcm.png',
 '6_IM-2192-1001.dcm.png',
 '7_IM-2263-1001.dcm.png',
 '8_IM-2333-1001.dcm.png',
 '9_IM-2407-1001.dcm.png',
 '10_IM-0002-2001.dcm.png',
 '11_IM-0067-1001.dcm.png',
 '12_IM-0133-1001.dcm.png',
 '13_IM-0198-1001.dcm.png',
 '14_IM-0256-1001.dcm.png',
 '15_IM-0324-1001.dcm.png',
 '17_IM-0460-2001.dcm.png',
 '18_IM-0520-1001.dcm.png',
 '19_IM-0583-1001.dcm.png',
 '20_IM-0653-1001.dcm.png',
 '22_IM-0810-1001.dcm.png',
 '23_IM-0879-1001.dcm.png',
 '24_IM-0949-1001.dcm.png',
 '25_IM-1024-2001.dcm.png',
 '26_IM-1090-1001.dcm.png',
 '27_IM-1168-2002.dcm.png',
 '28_IM-1231-1001.dcm.png',
 '30_IM-1385-1001.dcm.png',
 '32_IM-1511-1001.dcm.png',
 '33_IM-1576-13013.dcm.png',
 '34_IM-1644-1001.dcm.png',
 '35_IM-1704-1001.dcm.png',
 '36_IM-1776-1001.dcm.png',
 '37_IM-1847-0001-0002.dcm.png',
 '38_IM-1911-1001.dcm.png',
 '39_IM-1978-1001.dcm.png',
 '40_IM-2050-1001.dcm.png',
 '41_IM-2055-1001.d

In [71]:
len(train_img_names)

3000

In [72]:
def load_clean_captions(captions, dataset): 
    dataset_des = dict() 
    for key, caption in captions.items(): 
        if key in dataset: 
            dataset_des[key]= 'startseq ' + caption + ' endseq'
    return dataset_des 

train_descriptions = load_clean_captions(result_dict, train_img_names) 
# print(train_descriptions['1000268201_693b08cb0e'])
len(train_descriptions)
# train_descriptions

3000

In [73]:
base_model = InceptionV3(weights = 'imagenet') 
cnnmodel = Model(base_model.input, base_model.layers[-2].output)

In [74]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
def preprocess_img(img_path): 
	# inception v3 excepts img in 299 * 299 * 3 
	img = load_img(img_path, target_size = (299, 299)) # img = PIL image object 
	x = img_to_array(img) # x = img_to_array(img)  

	# Add one more dimension 
	x = np.expand_dims(x, axis = 0) #(1,299,299,3)
	x = preprocess_input(x) # normalization, resizing
	return x 

def encode(image): 
	image = preprocess_img(image) 
	vec = cnnmodel.predict(image) 
	vec = np.reshape(vec, (vec.shape[1])) #  if (1, N), it reshapes it to (N,).
	return vec 

In [23]:
encoding_train = {} 
print(len(train_images_paths))
i=1
for img in train_images_paths:
    print(i)
    i+=1
    encoding_train[img[len(images_path):]] = encode(img)

3000
1
1/1 [==============================] - 1s 934ms/step
2
1/1 [==============================] - 0s 63ms/step
3
1/1 [==============================] - 0s 65ms/step
4
1/1 [==============================] - 0s 61ms/step
5
1/1 [==============================] - 0s 62ms/step
6
1/1 [==============================] - 0s 87ms/step
7
1/1 [==============================] - 0s 66ms/step
8
1/1 [==============================] - 0s 62ms/step
9
1/1 [==============================] - 0s 61ms/step
10
1/1 [==============================] - 0s 59ms/step
11
1/1 [==============================] - 0s 60ms/step
12
1/1 [==============================] - 0s 62ms/step
13
1/1 [==============================] - 0s 60ms/step
14
1/1 [==============================] - 0s 78ms/step
15
1/1 [==============================] - 0s 59ms/step
16
1/1 [==============================] - 0s 60ms/step
17
1/1 [==============================] - 0s 60ms/step
18
1/1 [==============================] - 0s 59ms/step
19
1/1 [=====

In [24]:
encoding_test ={}
i=1
for img in test_images_paths:
    print(i)
    i+=1
    encoding_test[img[len(images_path):]] = encode(img)

1
1/1 [==============================] - 0s 63ms/step
2
1/1 [==============================] - 0s 90ms/step
3
1/1 [==============================] - 0s 62ms/step
4
1/1 [==============================] - 0s 71ms/step
5
1/1 [==============================] - 0s 65ms/step
6
1/1 [==============================] - 0s 63ms/step
7
1/1 [==============================] - 0s 63ms/step
8
1/1 [==============================] - 0s 63ms/step
9
1/1 [==============================] - 0s 63ms/step
10
1/1 [==============================] - 0s 62ms/step
11
1/1 [==============================] - 0s 65ms/step
12
1/1 [==============================] - 0s 62ms/step
13
1/1 [==============================] - 0s 62ms/step
14
1/1 [==============================] - 0s 62ms/step
15
1/1 [==============================] - 0s 63ms/step
16
1/1 [==============================] - 0s 62ms/step
17
1/1 [==============================] - 0s 62ms/step
18
1/1 [==============================] - 0s 62ms/step
19
1/1 [===========

In [25]:
encoding_val ={}
i=1
for img in val_images_paths:
    print(i)
    i+=1
    encoding_val[img[len(images_path):]] = encode(img)

1
1/1 [==============================] - 0s 74ms/step
2
1/1 [==============================] - 0s 64ms/step
3
1/1 [==============================] - 0s 61ms/step
4
1/1 [==============================] - 0s 82ms/step
5
1/1 [==============================] - 0s 74ms/step
6
1/1 [==============================] - 0s 74ms/step
7
1/1 [==============================] - 0s 74ms/step
8
1/1 [==============================] - 0s 65ms/step
9
1/1 [==============================] - 0s 61ms/step
10
1/1 [==============================] - 0s 63ms/step
11
1/1 [==============================] - 0s 61ms/step
12
1/1 [==============================] - 0s 63ms/step
13
1/1 [==============================] - 0s 63ms/step
14
1/1 [==============================] - 0s 60ms/step
15
1/1 [==============================] - 0s 63ms/step
16
1/1 [==============================] - 0s 62ms/step
17
1/1 [==============================] - 0s 63ms/step
18
1/1 [==============================] - 0s 64ms/step
19
1/1 [===========

In [75]:
# list of all training captions 
all_train_captions = [] 
for key, val in train_descriptions.items(): 
    all_train_captions.append(val) 
        
# consider only words which occur atleast 10 times 
vocabulary = vocab 
threshold = 10 # you can change this value according to your need 
word_counts = {} 
for cap in all_train_captions: 
	for word in cap.split(' '): 
		word_counts[word] = word_counts.get(word, 0) + 1
vocab = [word for word in word_counts if word_counts[word] >= threshold] 
len(vocab)
# word mapping to integers 
ixtoword = {} 
wordtoix = {} 

ix = 1
for word in vocab: 
	wordtoix[word] = ix 
	ixtoword[ix] = word 
	ix += 1
len(ixtoword) == len(wordtoix)
# find the maximum length of a description in a dataset 


True

In [76]:
lengths = [len(des.split()) for des in all_train_captions]


In [77]:
import numpy as np 

In [78]:
lengths = np.array(lengths)


max_length = 40
max_length

40

In [80]:
word_counts

{'startseq': 3000,
 'The': 3099,
 'cardiac': 209,
 'silhouette': 738,
 'and': 2473,
 'mediastinum': 477,
 'size': 1326,
 'are': 3836,
 'within': 1347,
 'normal': 1753,
 'limits.': 956,
 'There': 2015,
 'is': 3791,
 'no': 1310,
 'pulmonary': 853,
 'edema.': 111,
 'focal': 1541,
 'consolidation.': 354,
 'XXXX': 1756,
 'of': 2136,
 'a': 403,
 'pleural': 2163,
 'effusion.': 885,
 'evidence': 311,
 'pneumothorax.': 1007,
 'endseq': 3000,
 'Borderline': 11,
 'cardiomegaly.': 64,
 'Midline': 4,
 'sternotomy': 63,
 'XXXX.': 234,
 'Enlarged': 4,
 'arteries.': 6,
 'Clear': 50,
 'lungs.': 76,
 'Inferior': 1,
 'diffuse': 22,
 'bilateral': 120,
 'interstitial': 110,
 'alveolar': 74,
 'opacities': 145,
 'consistent': 75,
 'with': 529,
 'chronic': 66,
 'obstructive': 11,
 'lung': 493,
 'disease': 91,
 'bullous': 5,
 'emphysema.': 27,
 'irregular': 7,
 'in': 1218,
 'the': 2379,
 'left': 562,
 'apex,': 9,
 'that': 13,
 'could': 23,
 'represent': 69,
 'cavitary': 8,
 'lesion': 4,
 'apex.There': 1,
 'str

In [81]:
all_train_captions

['startseq The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax. endseq',
 'startseq Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX. endseq',
 'startseq There are diffuse bilateral interstitial and alveolar opacities consistent with chronic obstructive lung disease and bullous emphysema. There are irregular opacities in the left lung apex, that could represent a cavitary lesion in the left lung apex.There are streaky opacities in the right upper lobe, XXXX scarring. The cardiomediastinal silhouette is normal in size and contour. There is no pneumothorax or large pleural effusion. endseq',
 'startseq The cardiomediastinal silhouette and pulmonary vasculature are within normal limits. There is no pneumothorax or pleural effusion. There are no focal areas of consolid

In [82]:
len(wordtoix)

609

In [83]:
train_features = encoding_train

In [84]:
X1, X2, y = list(), list(), list() 
for key, caption in train_descriptions.items(): 
    pic = train_features[key] 
    # sequence of indices(corresponding integer index ) of a caption
    seq = [wordtoix[word] for word in caption.split(' ') if word in wordtoix]  
    for i in range(1, len(seq)): 
        # in_seq is the input sequence up to the i-th word.
        #out_seq is the i-th word, which is the next word in the sequence.
        in_seq, out_seq = seq[:i], seq[i] 
        in_seq = pad_sequences([in_seq], maxlen = max_length)[0] 
        out_seq = to_categorical([out_seq], num_classes = len(wordtoix) +1)[0] 
        # store 
        X1.append(pic)
        X2.append(in_seq) 
        y.append(out_seq) 
  
X2 = np.array(X2) 
X1 = np.array(X1) 
y = np.array(y) 

KeyError: '326_IM-1546-2002.dcm.png'

In [ ]:
embeddings_index = {} 
golve_path ='/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'
#  reads the entire contents of the file into a string.
glove = open(golve_path, 'r', encoding = 'utf-8').read() 
for line in glove.split("\n"): 
    values = line.split(" ") 
    word = values[0] 
    indices = np.asarray(values[1: ], dtype = 'float32') 
    embeddings_index[word] = indices 
  
emb_dim = 200
emb_matrix = np.zeros((len(wordtoix) +1, emb_dim)) 
for word, i in wordtoix.items(): 
    emb_vec = embeddings_index.get(word) 
    if emb_vec is not None: 
        emb_matrix[i] = emb_vec 
emb_matrix.shape

In [ ]:
emb_matrix[:10]

In [ ]:
val_img_names = [ imgname[len(images_path):] for imgname in val_images_paths]
val_img_names

In [ ]:
val_descriptions = load_clean_captions(result_dict, val_img_names) 
# print(train_descriptions['1000268201_693b08cb0e'])
len(val_descriptions)

In [ ]:
val_features = encoding_val

In [ ]:
X1_val, X2_val, y_val = list(), list(), list() 
for key, caption in val_descriptions.items(): 
    pic = val_features[key] 
    seq = [wordtoix[word] for word in caption.split(' ') if word in wordtoix] 
    for i in range(1, len(seq)): 
        in_seq, out_seq = seq[:i], seq[i] 
        in_seq = pad_sequences([in_seq], maxlen = max_length)[0] 
        out_seq = to_categorical([out_seq], num_classes = len(wordtoix) + 1)[0] 
        # store 
        X1_val.append(pic) 
        X2_val.append(in_seq) 
        y_val.append(out_seq) 
  
X2_val = np.array(X2_val) 
X1_val = np.array(X1_val) 
y_val = np.array(y_val) 

In [ ]:
encoding_train.keys()

In [ ]:
np.size(encoding_train['1_IM-0001-4001.dcm.png'])

In [ ]:

validation_data = ([X1_val, X2_val], y_val)

In [ ]:
vocab_size = len(wordtoix) + 1

In [ ]:
# define the model 
ip1 = Input(shape = (2048, )) 
ip2 = Input(shape = (max_length, )) 

fe1 = Dropout(0.2)(ip1) 
fe2 = Dense(256, activation = 'relu')(fe1) 

se1 = Embedding(vocab_size, emb_dim, mask_zero = True)(ip2) 
se2 = Dropout(0.2)(se1) 
se3 = LSTM(256, use_cudnn=False)(se2) 

decoder1 = add([fe2, se3]) 
decoder2 = Dense(256, activation = 'relu')(decoder1) 

outputs = Dense(vocab_size, activation = 'softmax')(decoder2) 
model = Model(inputs = [ip1, ip2], outputs = outputs)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
model.layers[2].set_weights([emb_matrix]) 
model.layers[2].trainable = True
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"]) 
model.fit([X1, X2], y, epochs = 50, batch_size = 64,validation_data=validation_data)

In [ ]:
def greedy_search2(pic):
    start = 'startseq'
    for i in range(max_length):
        seq = [wordtoix[word] for word in start.split() if word in wordtoix]
        seq = pad_sequences([seq], maxlen=max_length)
        yhat = model.predict([pic, seq])
        yhat = np.argmax(yhat,axis=-1)
        word = ixtoword[yhat[0]]
        start += ' ' + word
        if word == 'endseq':
            break
    final = start.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
encoding_test.keys()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# for key,val in encoding_test.items():
#     print(val.shape)
image_p = '3830_IM-1933-1001.dcm.png'
image_features = encoding_test[image_p].reshape(1,2048)
caption = greedy_search2(image_features)  # Generate caption using greedy search
print("Generated Caption:", caption)
# Assuming 'image_path' contains the path to the image file
image_path = images_path + image_p
print(result_dict[image_p])
# Load the image using Matplotlib
img = mpimg.imread(image_path)
# print("jhdkshfj",caption)